In [ ]:
from ortools.sat.python import cp_model

In [ ]:
tablero_m = [
    [3, 5, 4, 2, 1, 6],
    [3, 2, 5, 6, 4, 1],
    [2, 4, 3, 1, 5, 6],
    [5, 6, 4, 2, 3, 1],
    [2, 5, 3, 6, 4, 1],
    [1, 3, 4, 5, 6, 2]
]

tablero_w = [
    [2, 4, 5, 3, 6, 1],
    [3, 5, 4, 2, 1, 6],
    [1, 3, 6, 2, 4, 5],
    [3, 2, 5, 6, 4 ,1],
    [6, 4, 2, 1, 3, 5],
    [6, 4, 3, 1, 5, 2]
]


In [ ]:
model = cp_model.CpModel()

parejas = [[model.NewBoolVar(f"parejas_{m}_{w}") for w in range(6)] for m in range(6)]


n = 6

for m in range(6):
    model.Add(sum(parejas[m][w] for w in range(6)) == 1)
for w in range(6):
    model.Add(sum(parejas[m][w] for m in range(6)) == 1)


minRank = 4
for w in range(6):
    for m in range(6):
        if tablero_w[w][m] > minRank:
            model.Add(parejas[m][w] == 0) 

for m in range(6):
    for w in range(6):
        for m0 in range(6):
            for w0 in range(6):
                if tablero_m[m][w0] < tablero_m[m][w] and tablero_w[w0][m] < tablero_w[w0][m0]:
                    model.Add(parejas[m][w] + parejas[m0][w0] <= 1)

In [ ]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, parejas):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._parejas = parejas
        self.count = 0

    def OnSolutionCallback(self):
        self.count += 1
        matching = []
        for m in range(6):
            for w in range(6):
                if self.Value(self._parejas[m][w]) == 1:
                    matching.append((m, w))
                    break
       
        print(f"Solución {self.count}:")
        for m, w in matching:
            print(f"  Hombre {m+1}  —  Mujer {w+1}   (rank hombre: {tablero_m[m][w]}, rank mujer: {tablero_w[w][m]})")
        print("")

In [ ]:
solver = cp_model.CpSolver()


printer = SolutionPrinter(parejas)

solver.SearchForAllSolutions(model, printer)

print(f"Total soluciones encontradas: {printer.count}")
if printer.count == 0:
    print("No existe solucion.")

In [ ]:
possible_parejas = [(0,3), (1,4), (2,1), (3,5), (4,2), (5,0) ]


def validar(possible_parejas):
    for m, w in possible_parejas:
        for m0, w0 in possible_parejas:
            if m != m0 and w != w0:
                if tablero_w[m][w0] < tablero_w[m][w] and tablero_m[w0][m] < tablero_m[w0][m0]:
                    print(f"Pareja inestable: Hombre {m+1} con Mujer {w0+1} y Hombre {m0+1} con Mujer {w+1}")
                    return "INVALID"
            

    return "VALID"

resultado = validar(possible_parejas)

resultado